In [1]:
import h5py
import numpy as np
import os
from queue import Queue
import random

In [2]:
df = open('./Training Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]

In [3]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
chars_reverse = 'srqponmlkjihgfedcba'
coordinates_reverse = {k:v for v,k in enumerate(chars_reverse)}

coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

In [5]:
player_board = 0
oppnent_board = 1
player_air_1 = 2
player_air_2 = 3
player_air_3 = 4
player_air_4 = 5
oppnent_air_1 = 6
oppnent_air_2 = 7
oppnent_air_3 = 8
oppnent_air_4 = 9
empty_board = 10
last_1 = 11
round_3 = 12
round_5 = 13
round_7 = 14
round_9 = 15
round_11 = 16

In [6]:
liber = Queue()
def count_air2(move):
    bfs = []
    col = coordinates[move[2]]
    row = coordinates[move[3]]
    #找四周要bfs的
    
    if col - 1 >= 0 and x[row][col - 1][empty_board] == 0:
        bfs.append([row, col - 1])
    if col + 1 < 19 and x[row][col + 1][empty_board] == 0:
        bfs.append([row, col + 1])
    if row - 1 >= 0 and x[row - 1][col][empty_board] == 0:
        bfs.append([row - 1, col])
    if row + 1 < 19 and x[row + 1][col][empty_board] == 0:
        bfs.append([row + 1, col])
    bfs.append([row, col])
    for m in bfs:
        BFS(m)
    bfs = []
    if not liber.empty():
#         print("liber")
        while not liber.empty():
            li_row, li_col = liber.get()
#             x[li_row][li_col][8] = 1
            if li_col - 1 >= 0 and x[li_row][li_col - 1][empty_board] == 0:
                bfs.append([li_row, li_col - 1])
            if li_col + 1 < 19 and x[li_row][li_col + 1][empty_board] == 0:
                bfs.append([li_row, li_col + 1])
            if li_row - 1 >= 0 and x[li_row - 1][li_col][empty_board] == 0:
                bfs.append([li_row - 1, li_col])
            if li_row + 1 < 19 and x[li_row + 1][li_col][empty_board] == 0:
                bfs.append([li_row + 1, li_col])
        for m in bfs:
            BFS(m)
        
def BFS(now):
    q = Queue()
    row = now[0]
    col = now[1]
    q.put((row, col))
    visited = [[False for _ in range(19)] for _ in range(19)]
    nodes = []
    air = 0
    next_board = player_air_1
    board = player
    if oppnent[row, col] == 1:
        board = oppnent
        next_board = oppnent_air_1
    while not q.empty():
        m, n = q.get()
        if m < 0 or n < 0 or m >= 19 or n >= 19 or visited[m][n]:
            continue
        visited[m][n] = True
        if board[m, n] == 1:
            nodes.append([m, n])
            q.put((m - 1, n))
            q.put((m + 1, n))
            q.put((m, n - 1))
            q.put((m, n + 1))
        elif x[m, n, empty_board] == 1: #空地 -> 自由度+1
            air += 1
    dele = -1
    for i in range(player_air_1, player_air_1 + 8):
        if x[row][col][i] == 1:
            dele = i
            break
    air = min(air, 4)
    for node in nodes:
        n_row = node[0]
        n_col = node[1]
        #原本的改0，現在的氣數對應的棋盤改1
#         if dele != -1:
#             x[n_row][n_col][dele] = 0;
        for i in range(player_air_1, player_air_1 + 8):
            x[n_row][n_col][i] = 0;
        if air == 0:
            x[n_row][n_col][empty_board] = 1
            player[n_row, n_col] = 0
            oppnent[n_row, n_col] = 0
            liber.put(node)
        else:
            x[n_row][n_col][next_board + air - 1] = 1

In [8]:
x = np.zeros((19,19,17))
player = np.zeros((19, 19))
oppnent = np.zeros((19, 19))


def prepare_input(moves, player_color):
    sz = len(moves)
    move = moves[-1]
    
    color = move[0]
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    if color == player_color: #
        player[row,column] = 1
    else: #
        oppnent[row,column] = 1
    x[row,column,empty_board] = 0
    
    #最後1步
    x[:, :, last_1] = 0
    x[row, column, last_1] = 1
    
    #周圍
    x[:, :, round_3:round_11 + 1] = 0
    last_col = coordinates[move[2]]
    last_row = coordinates[move[3]]
    for rad in range(1, 6):
        row1 = max(0, last_row - rad)
        row7 = min(18, last_row + rad)
        col1 = max(0, last_col - rad)
        col7 = min(18, last_col + rad)
        for i in range(row1, row7 + 1, 1):
            for j in range(col1, col7 + 1, 1):
                x[i, j, round_3 + rad - 1] = 1
    
    count_air2(move)
    x[:, :, player_board] = player.copy()
    x[:, :, oppnent_board] = oppnent.copy()
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

def prepare_label_rotate90(move):
    column = coordinates[move[2]]
    row = coordinates_reverse[move[3]]
    return row*19+column


In [9]:
from IPython.display import clear_output

if os.path.isfile('./hdf5/style_dataset_aug16.hdf5'):
    os.remove('./hdf5/style_dataset_aug16.hdf5')
hdf5 = h5py.File('./hdf5/style_dataset_aug16.hdf5', 'a')
hdf5.create_dataset('Xs', (0, 19, 19, 17), maxshape=(None, 19, 19, 17), chunks=(10, 19, 19, 17)
                    ,dtype=np.float32, compression="lzf")
hdf5.create_dataset('Ys', (0, 1), maxshape=(None, 1)
                    ,dtype=np.float32, compression="lzf")
x_dataset = hdf5['Xs']
y_dataset = hdf5['Ys']

x_batch = [] # Initialize data batch
y_batch = []
batch_size = 16
print(f'total{len(games)}')
indexs = np.arange(len(games))
# np.random.seed()
np.random.shuffle(indexs)
# cnt=0
for game_i in indexs[:int(0.95 * len(games))]:
# for game_i, game in enumerate(games[:10]): # Iterate through games
        clear_output(wait=True)
        print(game_i, end = ' ')
        x = np.zeros((19,19,17))
        player = np.zeros((19, 19))
        oppnent = np.zeros((19, 19))
        x[:, :, empty_board] = np.ones((19, 19))
        moves_list = games[game_i].split(',')
        for count, move in enumerate(moves_list):
            pi = prepare_input(moves_list[:count + 1], moves_list[-1][0])
#             print(np.shape(np.array(pi)))
            if count == len(moves_list) - 1:
                x_batch.append(pi.copy())
#                 print(np.shape(np.array(x_batch)))
                y_batch.append([game_styles[game_i]])
#                 mirror
                pi_rot180 = pi[::-1, ::-1, :]
                x_batch.append(pi_rot180.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_rot90 = np.rot90(pi, 3)
                x_batch.append(pi_rot90.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_rot270 = np.rot90(pi, 1)
                x_batch.append(pi_rot270.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi[:, ::-1, :]
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi[::-1, :, :]
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi[::-1, :, :]
                pi_mirror =  np.rot90(pi_mirror, 1)
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi[:, ::-1, :]
                pi_mirror =  np.rot90(pi_mirror, 1)
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_color_copy = pi.copy()
                pi_color = pi_color_copy.copy()
                pi_color[:,:,player_board], pi_color[:,:,oppnent_board] = pi_color_copy[:,:,oppnent_board], pi_color_copy[:,:,player_board]
                for i in range(player_air_1, player_air_1 + 4):
                    pi_color[:,:,i], pi_color[:,:,i + 4] = pi_color_copy[:,:,i + 4], pi_color_copy[:,:,i]
                x_batch.append(pi_color.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_rot180 = pi_color[::-1, ::-1, :]
                x_batch.append(pi_rot180.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_rot90 = np.rot90(pi_color, 3)
                x_batch.append(pi_rot90.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_rot270 = np.rot90(pi_color, 1)
                x_batch.append(pi_rot270.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi_color[:, ::-1, :]
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi_color[::-1, :, :]
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi_color[::-1, :, :]
                pi_mirror =  np.rot90(pi_mirror, 1)
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
                
                pi_mirror = pi_color[:, ::-1, :]
                pi_mirror =  np.rot90(pi_mirror, 1)
                x_batch.append(pi_mirror.copy())
                y_batch.append([game_styles[game_i]])
            
                if len(x_batch) >= batch_size or (game_i == len(games) - 1 
                                        and (count == len(moves_list) - 1 or count == len(moves_list) - 2)): # Yield when reached batch size
                        x_dataset.resize(x_dataset.shape[0] + len(x_batch), axis = 0)
                        x_dataset[-len(x_batch):] = np.array(x_batch) #這個dataset是對的
                        y_dataset.resize(y_dataset.shape[0] + len(y_batch), axis = 0)
                        y_dataset[-len(y_batch):] = np.array(y_batch)
                        x_batch = []
                        y_batch = []
hdf5.close()

10980 

In [10]:
from IPython.display import clear_output

if os.path.isfile('./hdf5/style_aug16_val.hdf5'):
    os.remove('./hdf5/style_aug16_val.hdf5')
hdf5 = h5py.File('./hdf5/style_aug16_val.hdf5', 'a')
hdf5.create_dataset('Xs', (0, 19, 19, 17), maxshape=(None, 19, 19, 17), chunks=(10, 19, 19, 17)
                    ,dtype=np.float32, compression="lzf")
hdf5.create_dataset('Ys', (0, 1), maxshape=(None, 1)
                    ,dtype=np.float32, compression="lzf")
x_dataset = hdf5['Xs']
y_dataset = hdf5['Ys']

x_batch = [] # Initialize data batch
y_batch = []
batch_size = 1
print(f'total{len(games)}')
# indexs = np.arange(len(games))
# np.random.seed()
# cnt=0
for game_i in indexs[int(0.95 * len(games)):]:
# for game_i, game in enumerate(games[:10]): # Iterate through games
        clear_output(wait=True)
        print(game_i, end = ' ')
        x = np.zeros((19,19,17))
        player = np.zeros((19, 19))
        oppnent = np.zeros((19, 19))
        x[:, :, empty_board] = np.ones((19, 19))
        moves_list = games[game_i].split(',')
        for count, move in enumerate(moves_list):
            pi = prepare_input(moves_list[:count + 1], moves_list[-1][0])
#             print(np.shape(np.array(pi)))
            if count == len(moves_list) - 1:
                x_batch.append(pi.copy())
                y_batch.append([game_styles[game_i]])
            
                if len(x_batch) >= batch_size or (game_i == len(games) - 1 
                                        and (count == len(moves_list) - 1 or count == len(moves_list) - 2)): # Yield when reached batch size
                        x_dataset.resize(x_dataset.shape[0] + len(x_batch), axis = 0)
                        x_dataset[-len(x_batch):] = np.array(x_batch) #這個dataset是對的
                        y_dataset.resize(y_dataset.shape[0] + len(y_batch), axis = 0)
                        y_dataset[-len(y_batch):] = np.array(y_batch)
                        x_batch = []
                        y_batch = []
hdf5.close()

23254 

In [11]:
# hdf5 = h5py.File('./hdf5/test.hdf5', 'r')
# print(np.shape(hdf5['Xs']))
# print(hdf5['Xs'].shape[0])
xs = h5py.File("./hdf5/test.hdf5", "a")["Xs"]
ys = h5py.File("./hdf5/test.hdf5", "a")["Ys"]
# xs = hdf5.get('Xs')
# ys = hdf5.get('Ys')
# print(np.shape(hdf5))
# import random
# random.shuffle(xs)
# for y in ys:
#     print(y, end=' ')
import time
start = time.time()
cnt = 0
for m in range(12):
    print(f'y = {ys[m]}')
# m = 1
#     print(ys[m],end=' ')
#     cnt+=1
    for i in range(player_air_4, player_air_4+1):
        print('hdf5[Xs]', end = " ")
    #     xss = xs[:,:,:,i]
        print(i)
        for j in range(19):
            for k in range(19):
    #                 xs[m][j][k][i]
                print(int(xs[m][j][k][i]), end = ' ')#第0個應該要沒有東西啊
            print()
        print()
end = time.time()
print(cnt)
print(end - start)
# print(ys[1])
# print(x_testing)

y = [1.]
hdf5[Xs] 5
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

y = [1.]
hdf5[Xs] 5
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0

In [12]:
hdf5.close()

In [10]:
# import random
# import h5py
# hdf5 = h5py.File('./hdf5/kyu_dataset_copy.hdf5', 'a')
# random.seed(1)
# random.shuffle(hdf5['Xs'])
# print("xs done!")
# random.seed(1)
# random.shuffle(hdf5['Ys'])
# print("ys done!")
# # xs = hdf5.get('Xs')
# # for i in range(8, 9):
# #     print('hdf5[Xs]', end = " ")
# #     print(i)
# #     for j in range(19):
# #         for k in range(19):
# #             print(int(xs[1][j][k][i]), end = ' ')#第0個應該要沒有東西啊
# #         print()
# #     print()
# # random.seed(1)
# # random.shuffle(hdf5['Ys'])